In [340]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind_from_stats
import urllib.request

In [341]:
## saving the appropriate NYC Open Data URLs to variables
URL16 = 'https://data.cityofnewyork.us/api/views/8u86-bviy/rows.csv?accessType=DOWNLOAD'
URL15 = 'https://data.cityofnewyork.us/api/views/hypw-js3b/rows.csv?accessType=DOWNLOAD'
URL14 = 'https://data.cityofnewyork.us/api/views/jzst-u7j8/rows.csv?accessType=DOWNLOAD'
URL13 = 'https://data.cityofnewyork.us/api/views/rgfe-8y2z/rows.csv?accessType=DOWNLOAD'
URL12 = 'https://data.cityofnewyork.us/api/views/5gde-fmj3/rows.csv?accessType=DOWNLOAD'

## saving the appropriate label for BBL to variables for each year
BBL16 = 'BBL - 10 digits'
BBL15 = 'NYC Borough, Block and Lot (BBL)'
BBL14 = 'BBL'
BBL13 = 'BBL'
BBL12 = 'BBL'

## saving the appropriate label for Site EUI to variables for each year
EUI16 = 'Site EUI (kBtu/ft²)'
EUI15 = 'Site EUI (kBtu/ft²)'
EUI14 = 'Site EUI(kBtu/ft2)'
EUI13 = 'Site EUI(kBtu/ft2)'
EUI12 = 'Site EUI(kBtu/ft2)'


In [342]:
# merges two year's worth of LL84 data on BBL and returns a data frame listing 
# the Site EUI difference value for each BBL that contains data for both years  
def difference(URL1, URL2, BBL_name1, BBL_name2, EUI_name1, EUI_name2):
    # loading building energy benchmarking data into dataframes
    year1 = pd.read_csv(URL1)
    year2 = pd.read_csv(URL2)
    
    # replacing the 'Not Available' values with NaN
    year1.replace('Not Available', np.nan, inplace=True)
    year1.replace('See Primary BBL', np.nan, inplace=True)
    year1.replace('NA', np.nan, inplace=True)
    year2.replace('NA', np.nan, inplace=True)
    year2.replace('Not Available', np.nan, inplace=True)
    year2.replace('See Primary BBL', np.nan, inplace=True)
    
    # making sure Site EUI is a float
    year1[EUI_name1] = year1[EUI_name1].astype(float)
    year2[EUI_name2] = year2[EUI_name2].astype(float)    
    
    # removing records with unrealistic Site EUI values
    year1.drop(year1[year1[EUI_name1] > 300].index, inplace=True)
    year2.drop(year2[year2[EUI_name2] > 300].index, inplace=True)
    year1.drop(year1[year1[EUI_name1] < 0].index, inplace=True)
    year2.drop(year2[year2[EUI_name2] < 0].index, inplace=True)
            
    # dropping every column but BBL and Site Energy Use Intensity
    year1 = year1.filter(items=[BBL_name1, EUI_name1])
    year2 = year2.filter(items=[BBL_name2, EUI_name2])
    
    # forcing BBLs out of scientific notation and then saving as string
    # then storing length of BBL string in new column
    year1['length'] = year1[BBL_name1].astype(str).str.len()
    year2['length'] = year2[BBL_name2].astype(str).str.len()
    
    year1.dropna(inplace=True)
    year2.dropna(inplace=True)
    
    # Any record with more than 12 digits includes more than one tax lot in a report
    # This is a difficult set-up to work with, so I am deleting those rows outright
    year1.drop(year1[year1['length'] >12].index, inplace=True)
    year2.drop(year2[year2['length'] >12].index, inplace=True)
    
    year1[BBL_name1] = pd.to_numeric(year1[BBL_name1], errors='coerce')#.astype(int)
    year2[BBL_name2] = pd.to_numeric(year2[BBL_name2], errors='coerce')
    
    year1 = year1.rename(columns={BBL_name1: 'BBL1', EUI_name1: 'SiteEUI1'})
    year2 = year2.rename(columns={BBL_name2: 'BBL2', EUI_name2: 'SiteEUI2'})
        
    ##year1 = year1.rename(columns={[0]: 'BBL1'})#,[1]: 'SiteEUI1', [2]: 'length1'})
    ##year2 = year2.rename(columns={[0]: 'BBL2'})#, [1]: 'SiteEUI2', [2]: 'length2'})    
    
    # merging on BBL to create dataset with Site EUI values for preRCx and postRCx years
    ##pop = year1.merge(year2, left_on=BBL_name1, right_on=BBL_name2, how='outer')
    
    
    
    pop = year1.merge(year2, left_on='BBL1', right_on='BBL2', how='inner')
    ##pop = pd.merge(year1, year2, how='inner', on=['BBL1', 'BBL2'], )

    ##pop = pop.rename(columns={BBL_name1: 'BBL1', EUI_name1: 'SiteEUI1', BBL_name2: 'BBL2', EUI_name2: 'SiteEUI2'})
    
    #dropping length column
    #pop.drop(['length1', 'length2'], axis=1, inplace=True)
    
    # dropping all rows with NaN
    ##pop.dropna(inplace=True)
    
    # creating a "difference" column which shows the change in Site EUI from year 1 to year 2
    pop['difference'] = pop['SiteEUI2'] - pop['SiteEUI1']
    return pop.drop_duplicates()

In [343]:
# execute teh difference function for three sets of bracket years
pop12_14 = difference(URL12, URL14, BBL12, BBL14, EUI12, EUI14)
pop13_15 = difference(URL13, URL15, BBL13, BBL15, EUI13, EUI15)
pop14_16 = difference(URL14, URL16, BBL14, BBL16, EUI14, EUI16)

In [344]:
# renaming SiteEUI columns to be more descriptive
pop12_14.rename(index=str, columns={"SiteEUI1": "SiteEUI2012", "SiteEUI2": "SiteEUI2014"},  inplace = True)
pop13_15.rename(index=str, columns={"SiteEUI1": "SiteEUI2013", "SiteEUI2": "SiteEUI2015"},  inplace = True)
pop14_16.rename(index=str, columns={"SiteEUI1": "SiteEUI2014", "SiteEUI2": "SiteEUI2016"},  inplace = True)

In [345]:
# dropping length columns
pop12_14.drop(['length_x', 'length_y', 'BBL2'], axis=1, inplace=True)
pop13_15.drop(['length_x', 'length_y', 'BBL2'], axis=1, inplace=True)
pop14_16.drop(['length_x', 'length_y', 'BBL2'], axis=1, inplace=True)

In [346]:
pop12_14.shape

(9269, 4)

In [347]:
pop13_15.shape

(9309, 4)

In [348]:
pop14_16.shape

(9268, 4)

In [349]:
# load LL87 retro-commissioning data and ropping duplicate rows
RCx13 = pd.read_csv('2013RCxBBL.csv').drop_duplicates()
RCx14 = pd.read_csv('2014RCxBBL.csv').drop_duplicates()
RCx15 = pd.read_csv('2015RCxBBL.csv').drop_duplicates()

In [350]:
# create column showing that retro-commissioning occurred in 2013
RCx13['RCx13'] = 1
RCx14['RCx14'] = 1
RCx15['RCx15'] = 1

In [351]:
# convert BBL to numeric so the inner merge will work 
RCx14['BBL'] = pd.to_numeric(RCx14['BBL'], errors='coerce')

# merge with appropriate pop dataframe and drop any duplicate rows
pop12_14 = pop12_14.merge(RCx13, left_on='BBL1', right_on='BBL', how='outer')
pop13_15 = pop13_15.merge(RCx14, left_on='BBL1', right_on='BBL', how='outer')
pop14_16 = pop14_16.merge(RCx15, left_on='BBL1', right_on='BBL', how='outer')

In [352]:
pop12_14.shape

(9601, 6)

In [353]:
pop13_15.shape

(9554, 6)

In [354]:
pop14_16.shape

(9517, 6)

In [355]:
# sample df is made of records where RCx occurred
sample12_14 = pop12_14.dropna()
sample13_15 = pop13_15.dropna()
sample14_16 = pop14_16.dropna()

population12_14 = pop12_14[pop12_14.RCx13 != 1]
population13_15 = pop13_15[pop13_15.RCx14 != 1]
population14_16 = pop14_16[pop14_16.RCx15 != 1]

In [356]:
population12_14.shape

(8121, 6)

In [357]:
population13_15.shape

(8551, 6)

In [358]:
population14_16.shape

(8518, 6)

In [359]:
sample12_14_diff = sample12_14['difference']
sample13_15_diff = sample13_15['difference']
sample14_16_diff = sample14_16['difference']

In [360]:
sample12_14_diff.describe()

count    1148.000000
mean        7.197038
std        33.173268
min      -247.400000
25%        -2.600000
50%         7.150000
75%        17.625000
max       165.900000
Name: difference, dtype: float64

In [361]:
sample13_15_diff.describe()

count    758.000000
mean      -0.488127
std       32.960562
min     -184.600000
25%      -11.275000
50%       -0.750000
75%        7.450000
max      192.700000
Name: difference, dtype: float64

In [362]:
sample14_16_diff.describe()

count    750.000000
mean      -5.660133
std       31.844735
min     -177.800000
25%      -14.475000
50%       -6.200000
75%        2.600000
max      176.200000
Name: difference, dtype: float64

In [363]:
pop12_14_diff = pop12_14['difference']
pop13_15_diff = pop13_15['difference']
pop14_16_diff = pop14_16['difference']

In [364]:
pop12_14_diff.describe()

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/numpy/lib/function_base.py:3403: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


count    9269.000000
mean       10.310077
std        34.240756
min      -247.400000
25%              NaN
50%              NaN
75%              NaN
max       280.100000
Name: difference, dtype: float64

In [365]:
pop13_15_diff.describe()

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/numpy/lib/function_base.py:3403: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


count    9309.000000
mean        0.175013
std        31.619721
min      -233.400000
25%              NaN
50%              NaN
75%              NaN
max       224.300000
Name: difference, dtype: float64

In [366]:
pop14_16_diff.describe()

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/numpy/lib/function_base.py:3403: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


count    9268.000000
mean       -5.919465
std        29.840313
min      -269.800000
25%              NaN
50%              NaN
75%              NaN
max       251.400000
Name: difference, dtype: float64

In [367]:
# using scipy.stats.ttest_ind_from_stats
# arguments includ: ttest_ind_from_stats(mean1, std1, nobs1, mean2, std2, nobs2, equal_var=True)
ttest13 = ttest_ind_from_stats(pop12_14_diff.mean(), pop12_14_diff.std(), pop12_14_diff.count(), 
                             sample12_14_diff.mean(), sample12_14_diff.std(), sample12_14_diff.count())

ttest14 = ttest_ind_from_stats(pop13_15_diff.mean(), pop13_15_diff.std(), pop13_15_diff.count(), 
                             sample13_15_diff.mean(), sample13_15_diff.std(), sample13_15_diff.count())

ttest15 = ttest_ind_from_stats(pop14_16_diff.mean(), pop14_16_diff.std(), pop14_16_diff.count(), 
                             sample14_16_diff.mean(), sample14_16_diff.std(), sample14_16_diff.count())

In [368]:
ttest13

Ttest_indResult(statistic=2.9156138992046285, pvalue=0.0035574637610890753)

In [369]:
ttest14

Ttest_indResult(statistic=0.5534436708704239, pvalue=0.57997193526538182)

In [370]:
ttest15

Ttest_indResult(statistic=-0.22774133326603771, pvalue=0.81985198594378894)